In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

In [31]:
result = pd.read_csv("../temp/result.csv", index_col='UserID')
result

,rec
UserID,
10,"1396:0.816,1968:0.782,2987:0.766,2406:0.745,11..."
23,"1270:0.848,2571:0.843,1214:0.724,1213:0.712,11..."
33,"457:0.888,1036:0.838,1090:0.766,541:0.762,296:..."
36,"50:0.906,2321:0.844,39:0.830,318:0.799,1222:0...."
42,"2571:0.771,2028:0.750,2716:0.729,2115:0.717,16..."
...,...
6010,"1193:0.788,2640:0.787,480:0.787,733:0.775,1073..."
6011,"733:0.911,380:0.804,2640:0.787,3114:0.775,2716..."
6016,"912:0.812,1213:0.786,3160:0.726,50:0.726,1234:..."


In [32]:
# 使用电影名替换电影ID
movies_1 = pd.read_csv("../data/ml-1m/movies.dat",
                       sep="::",
                       header=None,
                       engine='python',
                       encoding='ISO8859')
movies_1.columns = ['MovieID','Title','Genres']
movies_1.index = movies_1['MovieID']
movies_1.drop(['MovieID'], axis=1, inplace=True)
movies_1

,Title,Genres
MovieID,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy
...,...,...
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama


In [33]:
movies_2 = movies_1.copy()
movies_2['year'] = 0
for (idx, row), r in zip(movies_1.iterrows(), range(40000)):
    movies_2.loc[idx, 'year'] = int(row['Title'][-5:-1])
    for genres in row['Genres'].split('|'):
        if genres not in movies_2.columns:
            movies_2[genres] = 0
        movies_2.loc[idx, genres] = 1
    movies_2.loc[idx, 'Title'] = row['Title'][:-7].split(",")[0].split("(")[0]
movies_2.drop(['Genres'], axis=1, inplace=True)
movies_2

,Title,year,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
MovieID,,,,,,,,,,,,,,,,,,,,
1,Toy Story,1995,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Jumanji,1995,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men,1995,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Waiting to Exhale,1995,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,Father of the Bride Part II,1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,Meet the Parents,2000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3949,Requiem for a Dream,2000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3950,Tigerland,2000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [34]:
genres_list =movies_2.columns[2:].to_list()
genres_list

['Animation',
 "Children's",
 'Comedy',
 'Adventure',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Sci-Fi',
 'Documentary',
 'War',
 'Musical',
 'Mystery',
 'Film-Noir',
 'Western']

In [35]:
result["rec_names"] = ""
for idx, row in result.iterrows():
    rec_list = []
    for mid_score in row['rec'].split(","):
        mid, score = mid_score.split(":")
        rec_list.append(movies_2.loc[int(mid), 'Title'])
    result.loc[idx, 'rec_names'] = ",".join(rec_list)
result

,rec,rec_names
UserID,,
10,"1396:0.816,1968:0.782,2987:0.766,2406:0.745,11...","Sneakers,Breakfast Club,Who Framed Roger Rabbi..."
23,"1270:0.848,2571:0.843,1214:0.724,1213:0.712,11...","Back to the Future,Matrix,Alien,GoodFellas,Rai..."
33,"457:0.888,1036:0.838,1090:0.766,541:0.762,296:...","Fugitive,Die Hard,Platoon,Blade Runner,Pulp Fi..."
36,"50:0.906,2321:0.844,39:0.830,318:0.799,1222:0....","Usual Suspects,Pleasantville,Clueless,Shawshan..."
42,"2571:0.771,2028:0.750,2716:0.729,2115:0.717,16...","Matrix,Saving Private Ryan,Ghostbusters,Indian..."
...,...,...
6010,"1193:0.788,2640:0.787,480:0.787,733:0.775,1073...","One Flew Over the Cuckoo's Nest,Superman,Juras..."
6011,"733:0.911,380:0.804,2640:0.787,3114:0.775,2716...","Rock,True Lies,Superman,Toy Story 2,Ghostbuste..."
6016,"912:0.812,1213:0.786,3160:0.726,50:0.726,1234:...","Casablanca,GoodFellas,Magnolia,Usual Suspects,..."


In [36]:
# 按照Rating的情况补全用户已经看过的信息
ratings = pd.read_csv("../temp/ratings_mini.csv")
ratings.head(3)

,UserID,MovieID,Rating,Timestamp
0,10,648,4,978224925
1,10,2628,3,978228408
2,10,1682,5,978226319


In [37]:
result['like'] = ''
result['dislike'] = ''
for idx, row in ratings.iterrows():
    if row['Rating'] == 5:
        like = result.loc[row['UserID'], 'like']
        if len(like)>0:
            result.loc[row['UserID'], 'like'] = like+","+movies_2.loc[row['MovieID'], 'Title']
        else:
            result.loc[row['UserID'], 'like'] = movies_2.loc[row['MovieID'], 'Title']
    elif row['Rating'] < 3:
        dislike = result.loc[row['UserID'], 'dislike']
        if len(dislike)>0:
            result.loc[row['UserID'], 'dislike'] = dislike+","+movies_2.loc[row['MovieID'], 'Title']
        else:
            result.loc[row['UserID'], 'dislike'] = movies_2.loc[row['MovieID'], 'Title']
result

,rec,rec_names,like,dislike
UserID,,,,
10,"1396:0.816,1968:0.782,2987:0.766,2406:0.745,11...","Sneakers,Breakfast Club,Who Framed Roger Rabbi...","Truman Show,Ghost,Apollo 13,Toy Story,Groundho...","Clerks,Mad Max 2"
23,"1270:0.848,2571:0.843,1214:0.724,1213:0.712,11...","Back to the Future,Matrix,Alien,GoodFellas,Rai...","Being John Malkovich,Groundhog Day,Star Wars: ...","American President,2001: A Space Odyssey,Mad M..."
33,"457:0.888,1036:0.838,1090:0.766,541:0.762,296:...","Fugitive,Die Hard,Platoon,Blade Runner,Pulp Fi...","Shining,Stand by Me,One Flew Over the Cuckoo's...","Animal House,Truman Show,Clerks,Rock,Dances wi..."
36,"50:0.906,2321:0.844,39:0.830,318:0.799,1222:0....","Usual Suspects,Pleasantville,Clueless,Shawshan...","Who Framed Roger Rabbit?,Mission: Impossible,S...",Star Trek VI: The Undiscovered Country
42,"2571:0.771,2028:0.750,2716:0.729,2115:0.717,16...","Matrix,Saving Private Ryan,Ghostbusters,Indian...","Star Wars: Episode I - The Phantom Menace,Grou...",
...,...,...,...,...
6010,"1193:0.788,2640:0.787,480:0.787,733:0.775,1073...","One Flew Over the Cuckoo's Nest,Superman,Juras...","Terminator 2: Judgment Day,Toy Story,Dances wi...",Big Lebowski
6011,"733:0.911,380:0.804,2640:0.787,3114:0.775,2716...","Rock,True Lies,Superman,Toy Story 2,Ghostbuste...","Aladdin,Terminator 2: Judgment Day,Toy Story,C...","Fargo,Predator,Clockwork Orange,Graduate,Arach..."
6016,"912:0.812,1213:0.786,3160:0.726,50:0.726,1234:...","Casablanca,GoodFellas,Magnolia,Usual Suspects,...","Wizard of Oz,Citizen Kane,African Queen,Fargo,...","Gone with the Wind,Erin Brockovich,Abyss,Preda..."


In [38]:
# 由此可以得到任何一个用户的情况
result.loc[58]

rec          50:0.884,47:0.872,1233:0.855,2916:0.826,1278:0...
rec_names    Usual Suspects,Seven ,Boat,Total Recall,Young ...
like         Being John Malkovich,Shining,One Flew Over the...
dislike      Star Trek VI: The Undiscovered Country,Star Tr...
Name: 58, dtype: object

In [39]:
result.to_csv("../temp/result_addnames.csv")

In [40]:
# 也可以得到任何一个用户对不同电影类型的喜好程度
result2 = result.copy()
for genre in genres_list:
    result2[genre] = 0
for idx, row in result2.iterrows():
    rec_list = []
    for mid_score in row['rec'].split(","):
        mid, score = mid_score.split(":")
        for genre in genres_list:
            result2.loc[idx, genre] += movies_2.loc[int(mid), genre]
        rec_list.append(movies_2.loc[int(mid), 'Title'])
    result2.loc[idx, 'rec_names'] = ",".join(rec_list)
result2

,rec,rec_names,like,dislike,Animation,Children's,Comedy,Adventure,Fantasy,Romance,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
UserID,,,,,,,,,,,,,,,,,,,,,
10,"1396:0.816,1968:0.782,2987:0.766,2406:0.745,11...","Sneakers,Breakfast Club,Who Framed Roger Rabbi...","Truman Show,Ghost,Apollo 13,Toy Story,Groundho...","Clerks,Mad Max 2",1,0,4,3,0,3,...,1,3,0,3,0,1,0,1,1,0
23,"1270:0.848,2571:0.843,1214:0.724,1213:0.712,11...","Back to the Future,Matrix,Alien,GoodFellas,Rai...","Being John Malkovich,Groundhog Day,Star Wars: ...","American President,2001: A Space Odyssey,Mad M...",0,1,1,4,0,1,...,1,3,1,6,0,2,1,0,0,0
33,"457:0.888,1036:0.838,1090:0.766,541:0.762,296:...","Fugitive,Die Hard,Platoon,Blade Runner,Pulp Fi...","Shining,Stand by Me,One Flew Over the Cuckoo's...","Animal House,Truman Show,Clerks,Rock,Dances wi...",0,1,3,3,2,1,...,1,3,0,2,0,2,0,0,1,0
36,"50:0.906,2321:0.844,39:0.830,318:0.799,1222:0....","Usual Suspects,Pleasantville,Clueless,Shawshan...","Who Framed Roger Rabbit?,Mission: Impossible,S...",Star Trek VI: The Undiscovered Country,1,1,3,1,0,2,...,2,2,0,1,0,1,1,0,0,0
42,"2571:0.771,2028:0.750,2716:0.729,2115:0.717,16...","Matrix,Saving Private Ryan,Ghostbusters,Indian...","Star Wars: Episode I - The Phantom Menace,Grou...",,0,0,2,2,0,0,...,0,3,2,5,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,"1193:0.788,2640:0.787,480:0.787,733:0.775,1073...","One Flew Over the Cuckoo's Nest,Superman,Juras...","Terminator 2: Judgment Day,Toy Story,Dances wi...",Big Lebowski,0,1,2,6,1,1,...,0,2,0,4,0,1,0,0,0,0
6011,"733:0.911,380:0.804,2640:0.787,3114:0.775,2716...","Rock,True Lies,Superman,Toy Story 2,Ghostbuste...","Aladdin,Terminator 2: Judgment Day,Toy Story,C...","Fargo,Predator,Clockwork Orange,Graduate,Arach...",1,1,5,5,0,1,...,0,2,1,3,0,0,0,0,0,1
6016,"912:0.812,1213:0.786,3160:0.726,50:0.726,1234:...","Casablanca,GoodFellas,Magnolia,Usual Suspects,...","Wizard of Oz,Citizen Kane,African Queen,Fargo,...","Gone with the Wind,Erin Brockovich,Abyss,Preda...",0,0,2,0,0,1,...,3,1,0,1,0,2,0,1,2,0


In [41]:
# 由此可以得到任何一个用户对电影类型的喜好
result2.loc[58].iloc[4:].sort_values(ascending=False)

Thriller       4
Drama          4
Adventure      3
Comedy         2
Film-Noir      2
Action         2
Crime          2
Sci-Fi         1
Mystery        1
War            1
Animation      1
Horror         1
Documentary    0
Children's     0
Musical        0
Romance        0
Fantasy        0
Western        0
Name: 58, dtype: object